## Ugrađivanja

U našem prethodnom primjeru radili smo s vektorima vreće riječi visoke dimenzionalnosti duljine `vocab_size`, te smo eksplicitno pretvarali vektore niskodimenzionalne pozicijske reprezentacije u rijetku one-hot reprezentaciju. Ova one-hot reprezentacija nije memorijski učinkovita, a osim toga, svaka se riječ tretira neovisno o drugima, tj. one-hot kodirani vektori ne izražavaju nikakvu semantičku sličnost između riječi.

U ovoj jedinici nastavit ćemo istraživati **News AG** skup podataka. Za početak, učitajmo podatke i preuzmimo neke definicije iz prethodne bilježnice.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Što je ugradnja?

Ideja **ugradnje** je predstavljati riječi pomoću nižedimenzionalnih gustih vektora, koji na neki način odražavaju semantičko značenje riječi. Kasnije ćemo raspraviti kako izgraditi smislene ugradnje riječi, ali za sada razmotrimo ugradnje kao način smanjenja dimenzionalnosti vektora riječi.

Dakle, sloj za ugradnju uzima riječ kao ulaz i proizvodi izlazni vektor određene veličine `embedding_size`. Na neki način, to je vrlo slično sloju `Linear`, ali umjesto da koristi vektor kodiran kao one-hot, može uzeti broj riječi kao ulaz.

Korištenjem sloja za ugradnju kao prvog sloja u našoj mreži, možemo prijeći s modela vreće riječi na model **vreće ugradnji**, gdje prvo pretvaramo svaku riječ u našem tekstu u odgovarajuću ugradnju, a zatim izračunavamo neku agregatnu funkciju nad svim tim ugradnjama, poput `sum`, `average` ili `max`.

![Slika koja prikazuje klasifikator s ugradnjom za pet riječi u nizu.](../../../../../translated_images/hr/embedding-classifier-example.b77f021a7ee67eee.webp)

Naša neuronska mreža za klasifikaciju započet će slojem za ugradnju, zatim slojem za agregaciju, i linearnim klasifikatorom na vrhu:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Rad s promjenjivom veličinom sekvenci varijabli

Zbog ove arhitekture, minibatchovi za našu mrežu moraju se kreirati na određeni način. U prethodnoj jedinici, kada smo koristili vreću riječi (bag-of-words), svi BoW tenzori u minibatchu imali su jednaku veličinu `vocab_size`, bez obzira na stvarnu duljinu naše tekstualne sekvence. Kada prijeđemo na ugrađivanje riječi (word embeddings), završit ćemo s promjenjivim brojem riječi u svakom uzorku teksta, a prilikom kombiniranja tih uzoraka u minibatcheve morat ćemo primijeniti neko popunjavanje (padding).

To se može učiniti korištenjem iste tehnike pružanja funkcije `collate_fn` izvoru podataka:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Treniranje klasifikatora ugrađivanja

Sada kada smo definirali odgovarajući dataloader, možemo trenirati model koristeći funkciju za treniranje koju smo definirali u prethodnoj jedinici:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Napomena**: Ovdje treniramo samo za 25 tisuća zapisa (manje od jednog punog epoha) radi uštede vremena, ali možete nastaviti s treniranjem, napisati funkciju za treniranje kroz nekoliko epoha i eksperimentirati s parametrom stope učenja kako biste postigli veću točnost. Trebali biste moći postići točnost od oko 90%.


### Sloj EmbeddingBag i prikaz sekvenci promjenjive duljine

U prethodnoj arhitekturi morali smo popuniti sve sekvence na istu duljinu kako bismo ih uklopili u minibatch. Ovo nije najučinkovitiji način za prikazivanje sekvenci promjenjive duljine - drugi pristup bio bi korištenje **offset** vektora, koji bi sadržavao pomake svih sekvenci pohranjenih u jednom velikom vektoru.

![Slika koja prikazuje prikaz sekvenci pomoću offseta](../../../../../translated_images/hr/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Napomena**: Na slici iznad prikazana je sekvenca znakova, ali u našem primjeru radimo sa sekvencama riječi. Međutim, osnovni princip prikazivanja sekvenci pomoću offset vektora ostaje isti.

Za rad s prikazom pomoću offseta koristimo sloj [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Sličan je sloju `Embedding`, ali kao ulaz uzima sadržajni vektor i offset vektor, a također uključuje sloj za prosjek koji može biti `mean`, `sum` ili `max`.

Evo izmijenjene mreže koja koristi `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Da bismo pripremili skup podataka za treniranje, moramo osigurati funkciju za konverziju koja će pripremiti vektor pomaka:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Imajte na umu da, za razliku od svih prethodnih primjera, naša mreža sada prihvaća dva parametra: vektor podataka i vektor pomaka, koji su različitih veličina. Slično tome, naš učitavač podataka također nam pruža 3 vrijednosti umjesto 2: i tekstualni i vektori pomaka pružaju se kao značajke. Stoga trebamo malo prilagoditi našu funkciju treniranja kako bismo to uzeli u obzir:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantičke Ugradnje: Word2Vec

U našem prethodnom primjeru, sloj za ugradnju modela naučio je mapirati riječi u vektorsku reprezentaciju, no ta reprezentacija nije imala puno semantičkog značenja. Bilo bi korisno naučiti takvu vektorsku reprezentaciju gdje bi slične riječi ili sinonimi odgovarali vektorima koji su blizu jedni drugima prema nekoj metričkoj udaljenosti (npr. euklidskoj udaljenosti).

Da bismo to postigli, potrebno je unaprijed obučiti naš model za ugradnju na velikoj zbirci teksta na specifičan način. Jedan od prvih pristupa za obuku semantičkih ugradnji naziva se [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Temelji se na dvije glavne arhitekture koje se koriste za stvaranje distribuirane reprezentacije riječi:

 - **Kontinuirana vreća riječi** (CBoW) — u ovoj arhitekturi model se obučava da predvidi riječ iz okolnog konteksta. Za zadani ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, cilj modela je predvidjeti $W_0$ na temelju $(W_{-2},W_{-1},W_1,W_2)$.
 - **Kontinuirani skip-gram** je suprotan CBoW-u. Model koristi okolni prozor kontekstualnih riječi kako bi predvidio trenutnu riječ.

CBoW je brži, dok je skip-gram sporiji, ali bolje predstavlja riječi koje se rjeđe pojavljuju.

![Slika koja prikazuje algoritme CBoW i Skip-Gram za pretvaranje riječi u vektore.](../../../../../translated_images/hr/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Za eksperimentiranje s Word2Vec ugradnjom unaprijed obučenom na Google News skupu podataka, možemo koristiti biblioteku **gensim**. Ispod nalazimo riječi najsličnije riječi 'neural':

> **Napomena:** Kada prvi put kreirate vektore riječi, njihovo preuzimanje može potrajati!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Također možemo izračunati vektorske ugradnje iz riječi, koje će se koristiti za treniranje modela klasifikacije (prikazujemo samo prvih 20 komponenti vektora radi jasnoće):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Sjajna stvar kod semantičkih ugradnji je da možete manipulirati vektorskim kodiranjem kako biste promijenili semantiku. Na primjer, možemo tražiti riječ čija bi vektorska reprezentacija bila što bliža riječima *kralj* i *žena*, a što udaljenija od riječi *muškarac*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Oba, CBoW i Skip-Grams, su "prediktivni" ugrađeni modeli, jer uzimaju u obzir samo lokalne kontekste. Word2Vec ne koristi globalni kontekst.

**FastText** se nadovezuje na Word2Vec tako što uči vektorske reprezentacije za svaku riječ i znakove n-grama pronađene unutar svake riječi. Vrijednosti tih reprezentacija se zatim prosječno izračunavaju u jedan vektor pri svakom koraku treniranja. Iako ovo dodaje puno dodatnih izračuna tijekom pred-treniranja, omogućuje ugrađenim modelima riječi da kodiraju informacije o podriječima.

Druga metoda, **GloVe**, koristi ideju matrice su-pojavljivanja i primjenjuje neuronske metode za dekompoziciju matrice su-pojavljivanja u izraženije i nelinearne vektore riječi.

Možete se poigrati s primjerom mijenjajući ugrađene modele na FastText i GloVe, budući da gensim podržava nekoliko različitih modela za ugrađivanje riječi.


## Korištenje unaprijed istreniranih ugradnji u PyTorchu

Možemo prilagoditi gornji primjer kako bismo unaprijed popunili matricu u našem sloju za ugradnju semantičkim ugradnjama, poput Word2Vec-a. Trebamo uzeti u obzir da se rječnici unaprijed istreniranih ugradnji i našeg tekstualnog korpusa vjerojatno neće podudarati, pa ćemo inicijalizirati težine za nedostajuće riječi nasumičnim vrijednostima:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Sada trenirajmo naš model. Imajte na umu da je vrijeme potrebno za treniranje modela znatno duže nego u prethodnom primjeru, zbog veće veličine sloja za ugradnju, a time i znatno većeg broja parametara. Također, zbog toga ćemo možda morati trenirati naš model na više primjera ako želimo izbjeći prenaučenost.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

U našem slučaju ne vidimo značajan porast točnosti, što je vjerojatno zbog prilično različitih vokabulara.  
Kako bismo prevladali problem različitih vokabulara, možemo koristiti jedno od sljedećih rješenja:  
* Ponovno trenirati word2vec model na našem vokabularu  
* Učitati naš skup podataka s vokabularom iz unaprijed treniranog word2vec modela. Vokabular koji se koristi za učitavanje skupa podataka može se specificirati tijekom učitavanja.  

Drugi pristup čini se jednostavnijim, posebno zato što PyTorch `torchtext` okvir sadrži ugrađenu podršku za ugrađivanja. Na primjer, možemo instancirati vokabular temeljen na GloVe-u na sljedeći način:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Učitani rječnik ima sljedeće osnovne operacije:
* `vocab.stoi` rječnik omogućuje pretvaranje riječi u njezin indeks u rječniku
* `vocab.itos` radi suprotno - pretvara broj u riječ
* `vocab.vectors` je niz vektora ugrađivanja, pa da bismo dobili ugrađivanje riječi `s`, trebamo koristiti `vocab.vectors[vocab.stoi[s]]`

Evo primjera manipulacije ugrađivanjima kako bismo demonstrirali jednadžbu **kind-man+woman = queen** (morao sam malo prilagoditi koeficijent da bi radilo):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Da bismo obučili klasifikator koristeći te ugrađene vektore, prvo moramo kodirati naš skup podataka koristeći GloVe vokabular:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Kao što smo vidjeli gore, svi vektorski ugrađaji pohranjeni su u matrici `vocab.vectors`. To čini izuzetno jednostavnim učitavanje tih težina u težine sloja za ugrađivanje pomoću jednostavnog kopiranja:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Sada obučimo naš model i provjerimo hoćemo li dobiti bolje rezultate:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Jedan od razloga zašto ne vidimo značajno povećanje točnosti je činjenica da neke riječi iz našeg skupa podataka nedostaju u unaprijed uvježbanom GloVe rječniku, te se stoga u biti ignoriraju. Kako bismo prevladali ovu činjenicu, možemo uvježbati vlastite ugradnje na našem skupu podataka.


## Kontekstualni Ugrađeni Vektori

Jedno od ključnih ograničenja tradicionalnih unaprijed istreniranih reprezentacija ugrađenih vektora poput Word2Vec-a je problem razlučivanja značenja riječi. Iako unaprijed istrenirani ugrađeni vektori mogu uhvatiti dio značenja riječi u kontekstu, svako moguće značenje riječi kodirano je u isti vektor. To može uzrokovati probleme u modelima koji dolaze nakon, budući da mnoge riječi, poput riječi 'play', imaju različita značenja ovisno o kontekstu u kojem se koriste.

Na primjer, riječ 'play' u ove dvije rečenice ima prilično različita značenja:
- Otišao sam na **predstavu** u kazalištu.
- John želi **igrati** se s prijateljima.

Unaprijed istrenirani ugrađeni vektori gore predstavljaju oba ova značenja riječi 'play' u istom vektoru. Kako bismo prevladali ovo ograničenje, trebamo izgraditi ugrađene vektore temeljene na **jezičnom modelu**, koji je istreniran na velikom korpusu teksta i *zna* kako se riječi mogu kombinirati u različitim kontekstima. Rasprava o kontekstualnim ugrađenim vektorima izlazi izvan okvira ovog vodiča, ali ćemo im se vratiti kada budemo govorili o jezičnim modelima u sljedećoj jedinici.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije proizašle iz korištenja ovog prijevoda.
